In [1]:
%pwd

'/home/milad/projects/medical-nlp-pipeline/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/home/milad/projects/medical-nlp-pipeline'

In [ ]:
import opendatasets as od

# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/datasets/prasad22/ca-independent-medical-review'
# Using opendatasets let's download the data sets
od.download(dataset)

In [ ]:
from dataclasses import dataclass
from pathlib import Path

# specifies the type of value related to the key in yaml file
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str